# 코스닥 종목 추가
# 거래대금 추가
# 평균 거래량

In [1]:
from pykrx import stock
import pandas as pd
import datetime
from tqdm import tqdm

# 최근 1년(365일) 데이터를 조회할지 설정
days_ago = 365

# 오늘 날짜 계산
today = datetime.datetime.today().strftime('%Y%m%d')

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI 전 종목 리스트 가져오기
kospi_tickers = stock.get_market_ticker_list(market="KOSPI")

# 일일 변동성 지수 및 월별 수익률을 저장할 리스트
volatility_data = []

# 각 종목의 시세 데이터를 조회하고 변동성 및 월별 수익률 계산
for ticker in tqdm(kospi_tickers, desc="Processing KOSPI stocks"):
    # 각 종목의 일별 시세 조회 (종가 기준)
    df = stock.get_market_ohlcv_by_date(start_date, today, ticker)

    # 시작 시점의 주가 (첫 번째 거래일의 종가)
    start_price = df['종가'].iloc[0]

    # 마지막 시점의 주가 (가장 최근 거래일의 종가)
    current_price = df['종가'].iloc[-1]

    # 종가 기준 일일 수익률 계산
    df['daily_return'] = df['종가'].pct_change()

    # 일일 변동성 계산 (표준편차)
    daily_volatility = round(df['daily_return'].std() * 100, 2)  # 변동성을 %로 변환하고 소수점 둘째 자리까지

    # 일일 변동성을 12개월로 나누기 위한 계산
    df['month'] = df.index.to_period('M')  # 날짜를 월 단위로 변환
    monthly_volatility = df.groupby('month')['daily_return'].std() * 100  # 월별 변동성 계산
    monthly_volatility_list = [round(x, 2) for x in monthly_volatility.tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_volatility_list) < 12:
        monthly_volatility_list.insert(0, 0)

    # 수익률 계산 (시작 주가와 마지막 주가 기준)
    total_return = round(((current_price / start_price) - 1) * 100, 2)  # 수익률을 %로 계산하고 소수점 둘째 자리까지

    # 티커명으로 종목명 가져오기
    company_name = stock.get_market_ticker_name(ticker)

    # 월별 수익률 계산 (12개월)
    monthly_data = df.groupby('month')['종가'].agg(['first', 'last'])

    # 월별 수익률 계산 (시가와 종가 차이로 계산)
    monthly_data['return'] = (monthly_data['last'] / monthly_data['first'] - 1) * 100

    # 12개월 수익률을 리스트에 저장 (가장 최근 달부터 과거로)
    monthly_return_list = [round(x, 2) for x in monthly_data['return'].tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_return_list) < 12:
        monthly_return_list.insert(0, 0)

    # 샤프 비율 계산 (수익률 / 변동성)
    monthly_sharpe_ratio_list = []
    for i in range(12):
        if monthly_volatility_list[i] == 0:  # 변동성이 0일 경우 NaN으로 처리
            monthly_sharpe_ratio_list.append(float('nan'))
        else:
            monthly_sharpe_ratio_list.append(round(monthly_return_list[i] / monthly_volatility_list[i], 2))

    # 변동성 정보 및 월별 수익률을 저장 (변동성 컬럼을 사이사이에 추가)
    volatility_data.append({
        'Ticker': ticker,
        'Name': company_name,
        'Start Price': round(start_price, 2),  # 시작 주가 소수점 둘째 자리까지
        'Current Price': round(current_price, 2),  # 현재 주가 소수점 둘째 자리까지
        'Total Return (%)': total_return,  # 총 수익률 (%)
        'Month 1 Return (%)': monthly_return_list[-1],  # 가장 최근 월의 수익률
        'Month 2 Return (%)': monthly_return_list[-2],
        'Month 3 Return (%)': monthly_return_list[-3],
        'Month 4 Return (%)': monthly_return_list[-4],
        'Month 5 Return (%)': monthly_return_list[-5],
        'Month 6 Return (%)': monthly_return_list[-6],
        'Month 7 Return (%)': monthly_return_list[-7],
        'Month 8 Return (%)': monthly_return_list[-8],
        'Month 9 Return (%)': monthly_return_list[-9],
        'Month 10 Return (%)': monthly_return_list[-10],
        'Month 11 Return (%)': monthly_return_list[-11],
        'Month 12 Return (%)': monthly_return_list[-12],  # 12개월 전의 수익률
        'Month 1 Volatility (%)': monthly_volatility_list[-1],  # 가장 최근 월의 변동성
        'Month 2 Volatility (%)': monthly_volatility_list[-2],
        'Month 3 Volatility (%)': monthly_volatility_list[-3],
        'Month 4 Volatility (%)': monthly_volatility_list[-4],
        'Month 5 Volatility (%)': monthly_volatility_list[-5],
        'Month 6 Volatility (%)': monthly_volatility_list[-6],
        'Month 7 Volatility (%)': monthly_volatility_list[-7],
        'Month 8 Volatility (%)': monthly_volatility_list[-8],
        'Month 9 Volatility (%)': monthly_volatility_list[-9],
        'Month 10 Volatility (%)': monthly_volatility_list[-10],
        'Month 11 Volatility (%)': monthly_volatility_list[-11],
        'Month 12 Volatility (%)': monthly_volatility_list[-12],
        'Month 1 Sharpe Ratio': monthly_sharpe_ratio_list[-1],  # 가장 최근 월의 샤프 비율
        'Month 2 Sharpe Ratio': monthly_sharpe_ratio_list[-2],
        'Month 3 Sharpe Ratio': monthly_sharpe_ratio_list[-3],
        'Month 4 Sharpe Ratio': monthly_sharpe_ratio_list[-4],
        'Month 5 Sharpe Ratio': monthly_sharpe_ratio_list[-5],
        'Month 6 Sharpe Ratio': monthly_sharpe_ratio_list[-6],
        'Month 7 Sharpe Ratio': monthly_sharpe_ratio_list[-7],
        'Month 8 Sharpe Ratio': monthly_sharpe_ratio_list[-8],
        'Month 9 Sharpe Ratio': monthly_sharpe_ratio_list[-9],
        'Month 10 Sharpe Ratio': monthly_sharpe_ratio_list[-10],
        'Month 11 Sharpe Ratio': monthly_sharpe_ratio_list[-11],
        'Month 12 Sharpe Ratio': monthly_sharpe_ratio_list[-12]  # 12개월 전의 샤프 비율
    })

# 변동성 데이터를 데이터프레임으로 변환
volatility_df = pd.DataFrame(volatility_data)

# 인덱스 초기화
volatility_df.reset_index(drop=True, inplace=True)

# DataFrame 출력
volatility_df


Processing KOSPI stocks: 100%|██████████| 959/959 [00:34<00:00, 28.03it/s]

     Ticker     Name  Start Price  Current Price  Total Return (%)  \
0    095570   AJ네트웍스         3960           4735             19.57   
1    006840    AK홀딩스        16750          13020            -22.27   
2    027410      BGF         3370           3530              4.75   
3    282330   BGF리테일       139600         110500            -20.85   
4    138930  BNK금융지주         7060           9060             28.33   
..      ...      ...          ...            ...               ...   
954  079980      휴비스         5080           2970            -41.54   
955  005010      휴스틸         4920           4155            -15.55   
956  000540     흥국화재         3200           3590             12.19   
957  000545    흥국화재우         5060           5530              9.29   
958  003280     흥아해운         1525           2245             47.21   

     Month 1 Return (%)  Month 2 Return (%)  Month 3 Return (%)  \
0                  0.74                2.52                3.24   
1                 -0.23  

# 수정 후

In [12]:
from pykrx import stock
import pandas as pd
import datetime
from tqdm import tqdm

# 최근 1년(365일) 데이터를 조회할지 설정
days_ago = 365

# 오늘 날짜 계산
today = datetime.datetime.today().strftime('%Y%m%d')

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI, KOSDAQ 종목 리스트 가져오기
kospi_tickers = stock.get_market_ticker_list(market="KOSPI")
kosdaq_tickers = stock.get_market_ticker_list(market="KOSDAQ")

# 모든 티커와 시장 정보 결합
tickers = [(ticker, 'KOSPI') for ticker in kospi_tickers] + [(ticker, 'KOSDAQ') for ticker in kosdaq_tickers]

# 일일 변동성 지수 및 월별 수익률을 저장할 리스트
volatility_data = []

# 각 종목의 시세 데이터를 조회하고 변동성 및 월별 수익률 계산
for ticker, market in tqdm(tickers, desc="Processing stocks"):
    # 각 종목의 일별 시세 조회 (종가 기준)
    df = stock.get_market_ohlcv_by_date(start_date, today, ticker)

    # 시작 시점의 주가 (첫 번째 거래일의 종가)
    start_price = df['종가'].iloc[0]

    # 마지막 시점의 주가 (가장 최근 거래일의 종가)
    current_price = df['종가'].iloc[-1]

    # 평균 거래량과 거래대금 계산
    avg_volume = df['거래량'].mean()
    avg_trading_value = df['거래대금'].mean()

    # 종가 기준 일일 수익률 계산
    df['daily_return'] = df['종가'].pct_change()

    # 일일 변동성 계산 (표준편차)
    daily_volatility = round(df['daily_return'].std() * 100, 2)  # 변동성을 %로 변환하고 소수점 둘째 자리까지

    # 일일 변동성을 12개월로 나누기 위한 계산
    df['month'] = df.index.to_period('M')  # 날짜를 월 단위로 변환
    monthly_volatility = df.groupby('month')['daily_return'].std() * 100  # 월별 변동성 계산
    monthly_volatility_list = [round(x, 2) for x in monthly_volatility.tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_volatility_list) < 12:
        monthly_volatility_list.insert(0, 0)

    # 수익률 계산 (시작 주가와 마지막 주가 기준)
    total_return = round(((current_price / start_price) - 1) * 100, 2)  # 수익률을 %로 계산하고 소수점 둘째 자리까지

    # 티커명으로 종목명 가져오기
    company_name = stock.get_market_ticker_name(ticker)

    # 월별 수익률 계산 (12개월)
    monthly_data = df.groupby('month')['종가'].agg(['first', 'last'])

    # 월별 수익률 계산 (시가와 종가 차이로 계산)
    monthly_data['return'] = (monthly_data['last'] / monthly_data['first'] - 1) * 100

    # 12개월 수익률을 리스트에 저장 (가장 최근 달부터 과거로)
    monthly_return_list = [round(x, 2) for x in monthly_data['return'].tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_return_list) < 12:
        monthly_return_list.insert(0, 0)

    # 샤프 비율 계산 (수익률 / 변동성)
    monthly_sharpe_ratio_list = []
    for i in range(12):
        if monthly_volatility_list[i] == 0:  # 변동성이 0일 경우 NaN으로 처리
            monthly_sharpe_ratio_list.append(float('nan'))
        else:
            monthly_sharpe_ratio_list.append(round(monthly_return_list[i] / monthly_volatility_list[i], 2))

    # 변동성 정보 및 월별 수익률을 저장 (변동성 컬럼을 사이사이에 추가)
    volatility_data.append({
        'Ticker': ticker,
        'Market': market,  # 시장 구분 컬럼 추가
        'Name': company_name,
        'Start Price': round(start_price, 2),  # 시작 주가 소수점 둘째 자리까지
        'Current Price': round(current_price, 2),  # 현재 주가 소수점 둘째 자리까지
        'Total Return (%)': total_return,  # 총 수익률 (%)
        'Avg Volume': round(avg_volume, 2),  # 평균 거래량 추가
        'Avg Trading Value': round(avg_trading_value, 2),  # 평균 거래대금 추가
        'Month 1 Return (%)': monthly_return_list[-1],  # 가장 최근 월의 수익률
        'Month 2 Return (%)': monthly_return_list[-2],
        'Month 3 Return (%)': monthly_return_list[-3],
        'Month 4 Return (%)': monthly_return_list[-4],
        'Month 5 Return (%)': monthly_return_list[-5],
        'Month 6 Return (%)': monthly_return_list[-6],
        'Month 7 Return (%)': monthly_return_list[-7],
        'Month 8 Return (%)': monthly_return_list[-8],
        'Month 9 Return (%)': monthly_return_list[-9],
        'Month 10 Return (%)': monthly_return_list[-10],
        'Month 11 Return (%)': monthly_return_list[-11],
        'Month 12 Return (%)': monthly_return_list[-12],  # 12개월 전의 수익률
        'Month 1 Volatility (%)': monthly_volatility_list[-1],  # 가장 최근 월의 변동성
        'Month 2 Volatility (%)': monthly_volatility_list[-2],
        'Month 3 Volatility (%)': monthly_volatility_list[-3],
        'Month 4 Volatility (%)': monthly_volatility_list[-4],
        'Month 5 Volatility (%)': monthly_volatility_list[-5],
        'Month 6 Volatility (%)': monthly_volatility_list[-6],
        'Month 7 Volatility (%)': monthly_volatility_list[-7],
        'Month 8 Volatility (%)': monthly_volatility_list[-8],
        'Month 9 Volatility (%)': monthly_volatility_list[-9],
        'Month 10 Volatility (%)': monthly_volatility_list[-10],
        'Month 11 Volatility (%)': monthly_volatility_list[-11],
        'Month 12 Volatility (%)': monthly_volatility_list[-12],
        'Month 1 Sharpe Ratio': monthly_sharpe_ratio_list[-1],  # 가장 최근 월의 샤프 비율
        'Month 2 Sharpe Ratio': monthly_sharpe_ratio_list[-2],
        'Month 3 Sharpe Ratio': monthly_sharpe_ratio_list[-3],
        'Month 4 Sharpe Ratio': monthly_sharpe_ratio_list[-4],
        'Month 5 Sharpe Ratio': monthly_sharpe_ratio_list[-5],
        'Month 6 Sharpe Ratio': monthly_sharpe_ratio_list[-6],
        'Month 7 Sharpe Ratio': monthly_sharpe_ratio_list[-7],
        'Month 8 Sharpe Ratio': monthly_sharpe_ratio_list[-8],
        'Month 9 Sharpe Ratio': monthly_sharpe_ratio_list[-9],
        'Month 10 Sharpe Ratio': monthly_sharpe_ratio_list[-10],
        'Month 11 Sharpe Ratio': monthly_sharpe_ratio_list[-11],
        'Month 12 Sharpe Ratio': monthly_sharpe_ratio_list[-12]  # 12개월 전의 샤프 비율
    })

# 변동성 데이터를 데이터프레임으로 변환
volatility_df = pd.DataFrame(volatility_data)

# 인덱스 초기화
volatility_df.reset_index(drop=True, inplace=True)

# DataFrame 출력
volatility_df


Processing stocks:   0%|          | 0/2714 [00:00<?, ?it/s]


KeyError: '거래대금'